In [1]:
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import AdaBoostClassifier

In [10]:
def entropy(y, weights=None):
    """
    Compute the entropy of a vector y by considering the counts of the unique values (v1, ... vk), in z. 
    Include the weights of the boosted examples if present

    Returns the entropy of z: H(z) = p(z=v1) log2(p(z=v1)) + ... + p(z=vk) log2(p(z=vk))
    """

    # INSERT YOUR CODE HERE
    if weights is None:
      weights = np.ones_like(y) / len(y)

    unique_labels, label_counts = np.unique(y, return_counts=True)
    label_weights = np.array([np.sum(weights[y == label]) for label in unique_labels])

    probabilities = label_weights / np.sum(weights)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy
    


def mutual_information(x, y, weights=None):
    """
    
    Compute the mutual information between a data column (x) and the labels (y). The data column is a single attribute
    over all the examples (n x 1). Mutual information is the difference between the entropy BEFORE the split set, and
    the weighted-average entropy of EACH possible split.

    Returns the mutual information: I(x, y) = H(y) - H(y | x)

    Compute the weighted mutual information for Boosted learners
    """

    # INSERT YOUR CODE HERE
    if weights is None:
      weights = np.ones_like(y) / len(y)

    # Calculate the entropy before splitting
    initial_entropy = entropy(y, weights)

    # Calculate the weighted average entropy after splitting
    unique_values, value_counts = np.unique(x, return_counts=True)
    conditional_entropy = 0

    for value in unique_values:
        value_indices = x == value
        y_subset = y[value_indices]
        weights_subset = weights[value_indices]

        value_weight = np.sum(weights_subset)
        value_entropy = entropy(y_subset, weights_subset)

        conditional_entropy += (value_weight / np.sum(weights)) * value_entropy

    # Calculate the mutual information
    mutual_info = initial_entropy - conditional_entropy
    return mutual_info
    


def id3(x, y, attribute_value_pairs=None, depth=0, max_depth=5, weights=None):
    """
    Implements the classical ID3 algorithm given training data (x), training labels (y) and an array of
    attribute-value pairs to consider. This is a recursive algorithm that depends on three termination conditions
        1. If the entire set of labels (y) is pure (all y = only 0 or only 1), then return that label
        2. If the set of attribute-value pairs is empty (there is nothing to split on), then return the most common
           value of y (majority label)
        3. If the max_depth is reached (pre-pruning bias), then return the most common value of y (majority label)
    Otherwise the algorithm selects the next best attribute-value pair using INFORMATION GAIN as the splitting criterion
    and partitions the data set based on the values of that attribute before the next recursive call to ID3.

    The tree we learn is a BINARY tree, which means that every node has only two branches. The splitting criterion has
    to be chosen from among all possible attribute-value pairs. That is, for a problem with two features/attributes x1
    (taking values a, b, c) and x2 (taking values d, e), the initial attribute value pair list is a list of all pairs of
    attributes with their corresponding values:
    [(x1, a),
     (x1, b),
     (x1, c),
     (x2, d),
     (x2, e)]
     If we select (x2, d) as the best attribute-value pair, then the new decision node becomes: [ (x2 == d)? ] and
     the attribute-value pair (x2, d) is removed from the list of attribute_value_pairs.

    The tree is stored as a nested dictionary, where each entry is of the form
                    (attribute_index, attribute_value, True/False): subtree
    * The (attribute_index, attribute_value) determines the splitting criterion of the current node. For example, (4, 2)
    indicates that we test if (x4 == 2) at the current node.
    * The subtree itself can be nested dictionary, or a single label (leaf node).
    * Leaf nodes are (majority) class labels

    Returns a decision tree represented as a nested dictionary, for example
    {(4, 1, False):
        {(0, 1, False):
            {(1, 1, False): 1,
             (1, 1, True): 0},
         (0, 1, True):
            {(1, 1, False): 0,
             (1, 1, True): 1}},
     (4, 1, True): 1}
    """

    # INSERT YOUR CODE HERE. NOTE: THIS IS A RECURSIVE FUNCTION.
    if weights is None:
      weights = np.ones_like(y) / len(y)

    if attribute_value_pairs is None:
        attribute_value_pairs = [(i, value) for i in range(x.shape[1]) for value in np.unique(x[:, i])]

    # Termination conditions
    if y is None:
      return 0
    if np.all(y == y[0]):  # If all labels are the same (pure node)
        return y[0]
    elif not attribute_value_pairs or depth == max_depth:  # If there is nothing to split on or max depth reached
        return np.argmax(np.bincount(y, weights=weights))  # Return the majority label

    # Select the best attribute-value pair using information gain
    gains = [mutual_information(x[:, i] == value, y, weights) for i, value in attribute_value_pairs]
    best_pair_index = np.argmax(gains)
    best_attribute, best_value = attribute_value_pairs[best_pair_index]

    # Split the dataset based on the best attribute-value pair
    x_left, y_left, w_left = x[x[:, best_attribute] == best_value], y[x[:, best_attribute] == best_value], weights[x[:, best_attribute] == best_value]
    x_right, y_right, w_right = x[x[:, best_attribute] != best_value], y[x[:, best_attribute] != best_value], weights[x[:, best_attribute] != best_value]

    # Remove the selected attribute-value pair from the list
    attribute_value_pairs = [pair for i, pair in enumerate(attribute_value_pairs) if i != best_pair_index]

    # Recursively build the decision tree on the partitioned datasets
    left_subtree = id3(x_left, y_left, attribute_value_pairs, depth+1, max_depth, w_left)
    right_subtree = id3(x_right, y_right, attribute_value_pairs, depth+1, max_depth, w_right)

    # Store the decision tree as a nested dictionary
    decision_tree = {
        (best_attribute, best_value, True): left_subtree,
        (best_attribute, best_value, False): right_subtree
    }

    return decision_tree
    

def bootstrap_sampler(x, y, num_samples):
    indices = np.random.choice(len(x), num_samples, replace=True)

    # Use the indices to select the corresponding data points from x and y
    x_sample = x[indices]
    y_sample = y[indices]

    return (x_sample, y_sample)
    


def bagging(x, y, max_depth, num_trees):
    """
    Implements bagging of multiple id3 trees where each tree trains on a boostrap sample of the original dataset
    """
    trees = []
    n_samples = x.shape[0]
    for i in range(num_trees):
        [x_sample, y_sample] = bootstrap_sampler(x, y, n_samples)

        # Train a decision tree on the bootstrap sample
        tree = id3(x_sample, y_sample, max_depth = max_depth)

        # Add the trained decision tree to the list
        trees.append(tree)

    return trees, [1] * len(trees)
    raise Exception('Bagging not yet implemented!')

def boosting(x, y, max_depth, num_stumps):

    """
    Implements an adaboost algorithm using the id3 algorithm as a base decision tree
    """

    n = len(y)
    weights = np.ones(n) / n
    stumps = []
    stump_weights = []

    for t in range(num_stumps):
        # Train a decision tree with the current weights
        stump = id3(x, y, max_depth=max_depth, weights=weights)

        # Calculate the error of the current stump
        stump_preds = [predict_example(x_i, stump) for x_i in x]
        stump_errors = [pred != y_i for pred, y_i in zip(stump_preds, y)]
        
        weighted_error = np.sum(weights * stump_errors)
        # Calculate the weight of the current stump
        stump_weight = 0.5 * np.log((1 - weighted_error) / weighted_error)

        # Update the weights of the examples
        for j in range(len(weights)):
          if stump_preds[j] != y[j]:
              weights[j] = weights[j] * np.exp(weighted_error)
          else:
              weights[j] = weights[j] * np.exp(-weighted_error)
        weights = weights / np.sum(weights)

        # Save the stump and its weight
        stumps.append(stump)
        stump_weights.append(stump_weight)

    return stumps, stump_weights
    raise Exception('Boosting not yet implemented!')

def predict_example(x, tree):
    """
    Predicts the classification label for a single example x using tree by recursively descending the tree until
    a label/leaf node is reached.

    Returns the predicted label of x according to tree
    """
    y_pred = -1

    while (True):
        if tree == 1:
            return 1
        if tree == 0:
            return 0

        branch1, branch2 = tree.keys()
        feature = branch1[0]
        value = branch1[1]

        if (x[feature] == value):

            tree = tree[(feature, value, True)]
        else:
            tree = tree[(feature, value, False)]

 

def predict_example_ens(x, h_ens):
    """
    Predicts the classification label for a single example x using a combination of weighted trees
    Returns the predicted label of x according to tree
    """

    # INSERT YOUR CODE HERE. NOTE: THIS IS A RECURSIVE FUNCTION.

    stumps, weights = h_ens
    prediction = 0
    res = np.array([predict_example(x, stump) for stump in stumps])

    for i in range(len(res)):
        if res[i] == 0:
          prediction = prediction - weights[i]
        else:
          prediction = prediction + weights[i]

    return prediction > 0

def compute_error(y_true, y_pred):
    """
    Computes the average error between the true labels (y_true) and the predicted labels (y_pred)

    Returns the error = (1/n) * sum(y_true != y_pred)
    """

    # INSERT YOUR CODE HERE
    n = len(y_true)
    errors = sum([y_true[i] != y_pred[i] for i in range(n)])
    return errors / n
    

def confusion_matrix(y_true, y_pred):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(y_true)):
        if y_true[i] == 1 and y_pred[i] == 1:
            tp += 1
        if y_true[i] == 1 and y_pred[i] == 0:
            fn += 1
        if y_true[i] == 0 and y_pred[i] == 0:
            tn += 1
        if y_true[i] == 0 and y_pred[i] == 1:
            fp += 1
    return [[tp, fn], [fp, tn]]

def visualize(tree, depth=0):
    """
    Pretty prints (kinda ugly, but hey, it's better than nothing) the decision tree to the console. Use print(tree) to
    print the raw nested dictionary representation.
    DO NOT MODIFY THIS FUNCTION!
    """

    if depth == 0:
        print('TREE')

    for index, split_criterion in enumerate(tree):
        sub_trees = tree[split_criterion]

        # Print the current node: split criterion
        print('|\t' * depth, end='')
        print('+-- [SPLIT: x{0} = {1}]'.format(split_criterion[0], split_criterion[1]))

        # Print the children
        if type(sub_trees) is dict:
            visualize(sub_trees, depth + 1)
        else:
            print('|\t' * (depth + 1), end='')
            print('+-- [LABEL = {0}]'.format(sub_trees))


In [11]:
import pandas as pd
from sklearn import preprocessing
# Load the training data
df = pd.read_csv('./adult.data', header= None)
label_encoder = preprocessing.LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

# remove the continuous column
del df[2]


In [12]:
df.head()

,0,1,3,4,5,6,7,8,9,10,11,12,13,14
0,22,7,9,12,4,1,1,4,1,25,0,39,39,0
1,33,6,9,12,2,4,0,4,1,0,0,12,39,0
2,21,4,11,8,0,6,1,4,1,0,0,39,39,0
3,36,4,1,6,2,6,0,2,1,0,0,39,39,0
4,11,4,9,12,2,10,5,2,0,0,0,39,5,0


In [13]:
from sklearn.model_selection import train_test_split
dataset = df.to_numpy()[:10000]

Y = dataset[:, -1]
X = dataset[:, 0:-1]

Xtrn, Xtst, ytrn, ytst = train_test_split(
    X, Y, test_size=0.3, random_state=42)

In [16]:
# Compute the test error
OriginalTree = id3(Xtrn, ytrn, max_depth= 10)
baggingtree = bagging(Xtrn, ytrn, 10, 20)
boostingtree = boosting(Xtrn, ytrn, 10, 40)

y_pred_tst = [predict_example(x, OriginalTree) for x in Xtst]
y_pred_bagging = [predict_example_ens(x, baggingtree) for x in Xtst]
y_pred_boosting = [predict_example_ens(x, boostingtree) for x in Xtst]

tst_err = compute_error(ytst, y_pred_tst)
tst_err_bagging = compute_error(ytst, y_pred_bagging)
tst_err_boosting = compute_error(ytst, y_pred_boosting)

print('Test Error = {0:4.2f}%.'.format(tst_err * 100))
print('Test Error for bagging = {0:4.2f}%.'.format(tst_err_bagging * 100))
print('Test Error for boosting = {0:4.2f}%.'.format(tst_err_boosting * 100))


<ipython-input-10-a7e1836db9f5>:192: RuntimeWarning: divide by zero encountered in double_scalars
  stump_weight = 0.5 * np.log((1 - weighted_error) / weighted_error)


Test Error = 7.41%.
Test Error for bagging = 2.31%.
Test Error for boosting = 7.41%.
